In [2]:
from langchain_groq import ChatGroq


In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.apple.com/en-in/details/200539849/software-development-engineer-in-test-is-t?team=SFTWR")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.





Software Development Engineer in Test - IS&T - Jobs at Apple (IN) 













AppleStoreMaciPadiPhoneWatchAirPodsTV & HomeEntertainmentAccessoriesSupport


0+


    Please enable Javascript in your browser for best experience.
  

Jobs at AppleOpen MenuClose MenuWork at AppleLife at AppleProfileSign In
Search
Software Development Engineer in Test - IS&THyderabad, Telangana, IndiaSoftware and ServicesSubmit CVBack to search resultsSummaryPosted: 
                Jun 23, 2025Role Number:200539849At Apple, new ideas have a way of becoming extraordinary products, services and customer experiences very quickly. Bring passion and dedication to your job, and there's no telling what you could accomplish. The people here at Apple don't just craft products - they build the kind of wonder that's revolutionised entire industries!. It's the diversity of those people and their ideas that encourages the innovation that runs through everything we do, from amazing technology to industry-leading en

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [14]:
from langchain_core.output_parsers import JsonOutputParser
res = chain_extract.invoke(input={'page_data':page_data})

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Development Engineer in Test - IS&T',
 'experience': '4 to 14 years',
 'skills': ['Java',
  'Python',
  'Swift',
  'Core Java',
  'Karate',
  'RestAssured',
  'Playwright/Selenium',
  'Appium/XCUI',
  'Maven/Gradle',
  'Git',
  'Agile development',
  'CI/CD integrations',
  'Jmeter/Gatling/Locust',
  'AWS/GCP',
  'Docker',
  'Kubernetes',
  'ServiceNow'],
 'description': "The IS&T team is looking for a passionate Software Quality Engineer to contribute to design, analysis and certification of our applications. The role involves interacting with multi-functional engineering and product teams to test features, perform root cause analysis, and drive the quality of customer's experience."}

In [15]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [13]:
links = collection.query(query_texts=['Experience  python'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Java',
 'Python',
 'Swift',
 'Core Java',
 'Karate',
 'RestAssured',
 'Playwright/Selenium',
 'Appium/XCUI',
 'Maven/Gradle',
 'Git',
 'Agile development',
 'CI/CD integrations',
 'Jmeter/Gatling/Locust',
 'AWS/GCP',
 'Docker',
 'Kubernetes',
 'ServiceNow']

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at x. x is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of x 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase x's portfolio: {link_list}
        Remember you are Mohan, BDE at x. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Engineer in Test - IS&T Solutions for Enhanced Quality Assurance

Dear Hiring Manager,

I came across the job description for a Software Development Engineer in Test - IS&T at your esteemed organization and was impressed by the role's focus on designing, analyzing, and certifying applications to drive customer experience quality. As a Business Development Executive at x, I'd like to introduce our company's capabilities in fulfilling your software testing and quality assurance needs.

At x, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts is well-versed in a range of technologies, including Java, Python, and Swift, which aligns perfectly with your job requirements. We have hands-on experience with testing frameworks like Karate, RestAssured, Playwright/Selenium, and Appium/XCUI, ensuring that our solutio